In [1]:
%%capture
!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git
!pip install datasets

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None # None for auto detection.
load_in_4bit = True # Use 4bit quantization to reduce memory usage.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "mistralai/Mistral-7B-v0.3",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2024.11.10: Fast Mistral patching. Transformers:4.46.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.14G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/157 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/137k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2024.11.10 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


<a name="Data"></a>
### Data Prep


In [4]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""


EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["Instruction"]  # This column contains the instruction
    inputs = examples["Text"]  # The text to be converted to a mind map
    outputs = examples["Mindmap"]  # The actual mind map JSON structure

    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Construct the prompt
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)

    return {"text": texts}
pass

import pandas as pd
from datasets import Dataset

# Load your dataset
data = pd.read_csv("mindmap_summarize_train.csv", encoding='ISO-8859-1')
# Convert it into a Hugging Face dataset
dataset = Dataset.from_pandas(data)
# Apply the formatting function
dataset = dataset.map(formatting_prompts_func, batched=True)

# Check the first few entries after the mapping
print("\nTransformed Dataset Preview:")

# Convert the Hugging Face dataset back to a Pandas DataFrame for better display
transformed_df = dataset.to_pandas()

# Select and rename the relevant columns for clarity
transformed_df = transformed_df[["Instruction", "Text", "Mindmap", "text"]]
transformed_df.columns = ["Instruction", "Original Text", "Mindmap JSON", "Prompt Format"]

# Display the first few rows of the transformed data
print(transformed_df.head())


Map:   0%|          | 0/335 [00:00<?, ? examples/s]


Transformed Dataset Preview:
                                         Instruction  \
0  Convert the following text into a structured J...   
1  Convert the following text into a structured J...   
2  Convert the following text into a structured J...   
3  Convert the following text into a structured J...   
4  Convert the following text into a structured J...   

                                       Original Text  \
0  Animals are diverse and fascinating creatures ...   
1  Tremendous amounts of data are generated daily...   
2  Big data can be defined in terms of 6 Vs: vol...   
3  IOT is today one of the main applications of B...   
4  Big data has revolutionized the healthcare sec...   

                                        Mindmap JSON  \
0  {"mindmap": [{"name": "Animals", "subnodes": [...   
1  {"mindmap": [{"name": "Big Data", "subnodes": ...   
2  {"mindmap": [{"name": "Big Data", "subnodes": ...   
3  {"mindmap": [{"name": "IOT", "subnodes": [{"na...   
4  {"mindmap": [

<a name="Train"></a>
### Train the model


In [8]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 100,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
    ),
)

Map (num_proc=2):   0%|          | 0/335 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [10]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 335 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 100
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,0.979100
2,1.102000
3,1.172300
4,0.994600
5,1.015900
6,1.081100
7,0.961800
8,0.908900
9,0.837500
10,0.928400


<a name="Inference"></a>
### Inference

In [12]:
# alpaca_prompt
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Convert the following text into a structured JSON mind map with parent node and logical nested subnodes:", # instruction
        "IOT is today one of the main applications of Big data. Because of high diversity of things, there is a great evolution of IOT applications doing machine-to-machine communication. IOT has made it possible to track and locate positions of vehicles with the help of sensor data, wireless adaptors and GPS. In this way, an enormous a measure of information from IOT is gathered and stored on constant premise in the large information frameworks.", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 512, use_cache = True)
tokenizer.batch_decode(outputs)

['<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nConvert the following text into a structured JSON mind map with parent node and logical nested subnodes:\n\n### Input:\nIOT is today one of the main applications of Big data. Because of high diversity of things, there is a great evolution of IOT applications doing machine-to-machine communication. IOT has made it possible to track and locate positions of vehicles with the help of sensor data, wireless adaptors and GPS. In this way, an enormous a measure of information from IOT is gathered and stored on constant premise in the large information frameworks.\n\n### Response:\n{"mindmap": [{"name": "IOT and Big Data", "subnodes": [{"name": "IOT", "subnodes": [{"name": "Main Application of Big Data", "subnodes": []}, {"name": "High Diversity of Things", "subnodes": []}, {"name": "Machine-to-Machine Communicat

In [17]:
# alpaca_prompt
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Summarize and shorten the following text :", # instruction
        "IOT is today one of the main applications of Big data. Because of high diversity of things, there is a great evolution of IOT applications doing machine-to-machine communication. IOT has made it possible to track and locate positions of vehicles with the help of sensor data, wireless adaptors and GPS. In this way, an enormous a measure of information from IOT is gathered and stored on constant premise in the large information frameworks.", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 512, use_cache = True)
tokenizer.batch_decode(outputs)

['<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nSummarize and shorten the following text :\n\n### Input:\nIOT is today one of the main applications of Big data. Because of high diversity of things, there is a great evolution of IOT applications doing machine-to-machine communication. IOT has made it possible to track and locate positions of vehicles with the help of sensor data, wireless adaptors and GPS. In this way, an enormous a measure of information from IOT is gathered and stored on constant premise in the large information frameworks.\n\n### Response:\nIOT is today one of the main applications of Big data. Because of high diversity of things, there is a great evolution of IOT applications doing machine-to-machine communication. IOT has made it possible to track and locate positions of vehicles with the help of sensor data, wireless adaptors and

In [13]:
# alpaca_prompt
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Convert the following text into a structured JSON mind map with parent node and logical nested subnodes:", # instruction
        "Artificial neural networks (ANN). ANN model developed based on biological neurons’ analogy in human brains. One of the most straightforward ANN architectures is the perceptron which can be trained to predict based on a fixed threshold. ANN model can allow many inputs to pass through multiple processing layers to produce many outputs, so ANN can work with complex data to solve various classification and regression tasks.", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 512, use_cache = True)
tokenizer.batch_decode(outputs)

['<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nConvert the following text into a structured JSON mind map with parent node and logical nested subnodes:\n\n### Input:\nArtificial neural networks (ANN). ANN model developed based on biological neurons’ analogy in human brains. One of the most straightforward ANN architectures is the perceptron which can be trained to predict based on a fixed threshold. ANN model can allow many inputs to pass through multiple processing layers to produce many outputs, so ANN can work with complex data to solve various classification and regression tasks.\n\n### Response:\n{"mindmap": [{"name": "Artificial Neural Networks (ANN)", "subnodes": [{"name": "Developed based on Biological Neurons", "subnodes": []}, {"name": "Perceptron", "subnodes": [{"name": "Trained to Predict", "subnodes": []}, {"name": "Fixed Threshold", "su

In [18]:
# alpaca_prompt
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Summarize and shorten the following text :", # instruction
        "Artificial neural networks (ANN). ANN model developed based on biological neurons’ analogy in human brains. One of the most straightforward ANN architectures is the perceptron which can be trained to predict based on a fixed threshold. ANN model can allow many inputs to pass through multiple processing layers to produce many outputs, so ANN can work with complex data to solve various classification and regression tasks.", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 512, use_cache = True)
tokenizer.batch_decode(outputs)

['<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nSummarize and shorten the following text :\n\n### Input:\nArtificial neural networks (ANN). ANN model developed based on biological neurons’ analogy in human brains. One of the most straightforward ANN architectures is the perceptron which can be trained to predict based on a fixed threshold. ANN model can allow many inputs to pass through multiple processing layers to produce many outputs, so ANN can work with complex data to solve various classification and regression tasks.\n\n### Response:\nArtificial neural networks (ANN) are developed based on biological neuronsÂ\x92 analogy in human brains. ANN model can allow many inputs to pass through multiple processing layers to produce many outputs. ANN can work with complex data to solve various classification and regression tasks.</s>']

In [14]:
# alpaca_prompt
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Convert the following text into a structured JSON mind map with parent node and logical nested subnodes:", # instruction
        "In recent years, machine learning has become widely used in various applications and research. It plays a crucial role in numerous fields such as medical science and the healthcare system. In such scenarios, machine learning is used to diagnose sizeable medical data patterns or predict diseases. This paper presents a survey about different machine learning algorithms with their applications in various domains and shows the advantages of machine learning techniques that help create efficient support infrastructure for medical fields and improve healthcare services. This survey’s main objective is to highlight the previous work of machine learning algorithms implemented in the healthcare system and provide all necessary information to the researchers who want to explore machine learning in the healthcare system.", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 512, use_cache = True)
tokenizer.batch_decode(outputs)

['<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nConvert the following text into a structured JSON mind map with parent node and logical nested subnodes:\n\n### Input:\nIn recent years, machine learning has become widely used in various applications and research. It plays a crucial role in numerous fields such as medical science and the healthcare system. In such scenarios, machine learning is used to diagnose sizeable medical data patterns or predict diseases. This paper presents a survey about different machine learning algorithms with their applications in various domains and shows the advantages of machine learning techniques that help create efficient support infrastructure for medical fields and improve healthcare services. This survey’s main objective is to highlight the previous work of machine learning algorithms implemented in the healthcare 

In [19]:
# alpaca_prompt
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Summarize and shorten the following text :", # instruction
        "In recent years, machine learning has become widely used in various applications and research. It plays a crucial role in numerous fields such as medical science and the healthcare system. In such scenarios, machine learning is used to diagnose sizeable medical data patterns or predict diseases. This paper presents a survey about different machine learning algorithms with their applications in various domains and shows the advantages of machine learning techniques that help create efficient support infrastructure for medical fields and improve healthcare services. This survey’s main objective is to highlight the previous work of machine learning algorithms implemented in the healthcare system and provide all necessary information to the researchers who want to explore machine learning in the healthcare system.", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 512, use_cache = True)
tokenizer.batch_decode(outputs)

["<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nSummarize and shorten the following text :\n\n### Input:\nIn recent years, machine learning has become widely used in various applications and research. It plays a crucial role in numerous fields such as medical science and the healthcare system. In such scenarios, machine learning is used to diagnose sizeable medical data patterns or predict diseases. This paper presents a survey about different machine learning algorithms with their applications in various domains and shows the advantages of machine learning techniques that help create efficient support infrastructure for medical fields and improve healthcare services. This survey’s main objective is to highlight the previous work of machine learning algorithms implemented in the healthcare system and provide all necessary information to the researcher

In [15]:
# alpaca_prompt
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Convert the following text into a structured JSON mind map with parent node and logical nested subnodes:", # instruction
        "Semi-supervised learning is a combination of supervised and unsupervised machine learning techniques. These algorithms work with the dataset containing a small size of labeled data and a large size of unlabeled data, which provides both unsupervised and supervised learning advantages. It uses an unsupervised learning algorithm to cluster similar data, then uses current labeled data to label the remaining unlabeled data.", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 512)

<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Convert the following text into a structured JSON mind map with parent node and logical nested subnodes:

### Input:
Semi-supervised learning is a combination of supervised and unsupervised machine learning techniques. These algorithms work with the dataset containing a small size of labeled data and a large size of unlabeled data, which provides both unsupervised and supervised learning advantages. It uses an unsupervised learning algorithm to cluster similar data, then uses current labeled data to label the remaining unlabeled data.

### Response:
{"mindmap": [{"name": "Semi-Supervised Learning", "subnodes": [{"name": "Combination of Supervised and Unsupervised Learning", "subnodes": []}, {"name": "Dataset", "subnodes": [{"name": "Small Size of Labeled Data", "subnodes": []}, {"name": "Large Size of Unlabel

In [20]:
# alpaca_prompt
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Summarize and shorten the following text :", # instruction
        "Semi-supervised learning is a combination of supervised and unsupervised machine learning techniques. These algorithms work with the dataset containing a small size of labeled data and a large size of unlabeled data, which provides both unsupervised and supervised learning advantages. It uses an unsupervised learning algorithm to cluster similar data, then uses current labeled data to label the remaining unlabeled data.", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 512)

<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Summarize and shorten the following text :

### Input:
Semi-supervised learning is a combination of supervised and unsupervised machine learning techniques. These algorithms work with the dataset containing a small size of labeled data and a large size of unlabeled data, which provides both unsupervised and supervised learning advantages. It uses an unsupervised learning algorithm to cluster similar data, then uses current labeled data to label the remaining unlabeled data.

### Response:
Semi-supervised learning is a combination of supervised and unsupervised machine learning techniques. These algorithms work with the dataset containing a small size of labeled data and a large size of unlabeled data. It uses an unsupervised learning algorithm to cluster similar data, then uses current labeled data to label t

In [22]:
# alpaca_prompt
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Summarize and shorten the following text :", # instruction
        "Machine learning (ML) has transformed various industries by introducing intelligent algorithms capable of learning from data without explicit programming. Over the years, advancements in ML have led to the development of diverse techniques, including supervised, unsupervised, and reinforcement learning, each addressing unique challenges. Supervised learning, one of the most commonly used approaches, involves labeled datasets to train models for specific tasks like classification and regression. Algorithms like Support Vector Machines (SVM), Random Forests, and Neural Networks are prominent in this domain, solving problems ranging from spam detection in emails to predicting housing prices. On the other hand, unsupervised learning deals with unlabeled data, making it a powerful tool for tasks such as clustering, dimensionality reduction, and anomaly detection. Popular algorithms in this category, like K-Means Clustering and Principal Component Analysis (PCA), reveal hidden patterns in data, aiding industries such as retail and healthcare in understanding customer behavior or detecting rare medical conditions. Reinforcement learning, a more dynamic technique, focuses on decision-making processes where an agent learns optimal actions by interacting with an environment and receiving feedback in the form of rewards or penalties. It has revolutionized fields such as robotics, autonomous driving, and game development. Deep reinforcement learning, an extension of this approach, integrates neural networks to solve complex problems requiring large-scale computations. Artificial Neural Networks (ANNs), inspired by the human brain’s structure, have been instrumental in advancing ML capabilities. With architectures like Convolutional Neural Networks (CNNs) and Recurrent Neural Networks (RNNs), ANNs have enabled breakthroughs in image recognition, natural language processing (NLP), and time-series analysis. CNNs excel in processing visual data by identifying spatial hierarchies, while RNNs are tailored for sequential data, making them ideal for applications like speech recognition and stock price prediction.", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 512)

<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Summarize and shorten the following text :

### Input:
Machine learning (ML) has transformed various industries by introducing intelligent algorithms capable of learning from data without explicit programming. Over the years, advancements in ML have led to the development of diverse techniques, including supervised, unsupervised, and reinforcement learning, each addressing unique challenges. Supervised learning, one of the most commonly used approaches, involves labeled datasets to train models for specific tasks like classification and regression. Algorithms like Support Vector Machines (SVM), Random Forests, and Neural Networks are prominent in this domain, solving problems ranging from spam detection in emails to predicting housing prices. On the other hand, unsupervised learning deals with unlabeled data,

<a name="Save"></a>
### Saving, loading finetuned models


In [16]:
# Local saving
model.save_pretrained("lora_model")
tokenizer.save_pretrained("lora_model")

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.model',
 'lora_model/added_tokens.json',
 'lora_model/tokenizer.json')

In [21]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Save model and tokenizer to Google Drive
drive_save_path = '/content/drive/My Drive/mistralai_Mistral_v3_7b/Finetuned-For-Mindmaps-and-Summarization'
trainer.model.save_pretrained(drive_save_path)
tokenizer.save_pretrained(drive_save_path)

# Verify the save
import os
print("Files in the save directory:", os.listdir(drive_save_path))

Mounted at /content/drive
Files in the save directory: ['README.md', 'adapter_model.safetensors', 'adapter_config.json', 'tokenizer_config.json', 'special_tokens_map.json', 'tokenizer.model', 'tokenizer.json']


In [ ]:
max_seq_length = 2048
dtype = None
load_in_4bit = True

if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# alpaca_prompt

inputs = tokenizer(
[
    alpaca_prompt.format(
        "Convert the following text into a structured JSON mind map with parent node and logical nested subnodes:", # instruction
        "DL methods are different from standard ML algorithms because they do not need human intervention. They contain many hidden layers, including convolutional neural networks (CNN) and recurrent neural networks (RNN). These layers make it define the best features on unseen data. DL algorithms recently make outperformance in several challenging tasks like text understanding and identifying images and voice.", # input
        "", # output - leave this blank for generation!
    ),
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 512, use_cache = True)
tokenizer.batch_decode(outputs)

['<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nConvert the following text into a structured JSON mind map with parent node and logical nested subnodes:\n\n### Input:\nDL methods are different from standard ML algorithms because they do not need human intervention. They contain many hidden layers, including convolutional neural networks (CNN) and recurrent neural networks (RNN). These layers make it define the best features on unseen data. DL algorithms recently make outperformance in several challenging tasks like text understanding and identifying images and voice.\n\n### Response:\n{"mindmap": [{"name": "Deep Learning (DL)", "subnodes": [{"name": "Different from Standard ML Algorithms", "subnodes": [{"name": "No Human Intervention Required", "subnodes": []}]}, {"name": "Characteristics", "subnodes": [{"name": "Many Hidden Layers", "subnodes": []}, 